In [8]:
import torch
import os 
import numpy as np

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Robot Learning/HW1'

In [6]:
import os
import sys

GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)

files = os.listdir(GOOGLE_DRIVE_PATH)
expected_files = [ 'ROB498_hw1.ipynb', 'pytorch_intro.py', 'hands_on_regression.py', 'regression_training_data.npz', 'regression_validation_data.npz',  'robot_kinematics_regression.py', 'robot_kinematics_training_data.npz', 'robot_kinematics_validation_data.npz',  ]
print(f'Files found: {files}')

sys.path.append(GOOGLE_DRIVE_PATH)

# Verify that there are all the expected files in the directory
all_found = True
for expected_file in expected_files:
  if expected_file not in files:
    print(f'Required file {expected_file} not found!')
    all_found = False
if all_found:
  print('All required files are found :)')

Files found: ['Copy of sample_test_rob_learning.ipynb', 'robot_kinematics_regression.py', 'regression_training_data.npz', 'robot_kinematics_training_data.npz', 'regression_validation_data.npz', 'robot_kinematics_validation_data.npz', 'hands_on_regression.py', 'Using_Google_Colab.pdf', 'resources', '__pycache__', 'pytorch_intro.py', 'ROB498_hw1.ipynb']
All required files are found :)


In [9]:
train_data_path = os.path.join(GOOGLE_DRIVE_PATH, 'regression_training_data.npz')
val_data_path = os.path.join(GOOGLE_DRIVE_PATH, 'regression_validation_data.npz')

train_data = np.load(train_data_path)
val_data = np.load(val_data_path)

# Unpack the data
xs = torch.from_numpy(train_data['x'])[:, None] # shape (N, 1)
ys = torch.from_numpy(train_data['y'])[:, None] # shape (N, 1)

xs_val = torch.from_numpy(val_data['x'])[:, None] # shape (N, 1)
ys_val = torch.from_numpy(val_data['y'])[:, None] # shape (N, 1)

In [20]:
print(xs.shape)
print(xs[0])

torch.Size([100, 1])
tensor([-2.2820])


In [22]:
print(ys.shape)

torch.Size([100, 1])


In [17]:
degree = 5
# inputs are xs and int variable d for degree output [1,x,x**2,.....x**d]
one_col = torch.ones((xs.shape[0],1),dtype = xs.dtype)
x_d = torch.cat([one_col,xs],axis=1)
for i in range(1,degree):
  x_pow = xs.pow(i+1)
  x_d = torch.cat([x_d,x_pow],axis=1)

print(x_d.shape)
print(x_d[0])

torch.Size([100, 6])
tensor([  1.0000,  -2.2820,   5.2074, -11.8830,  27.1165, -61.8789])


In [25]:
xdx_inv = (x_d.T.mm(x_d)).inverse()
xdy = x_d.T.mm(ys)
coeffs = xdx_inv.mm(xdy)

In [27]:
coeffs = coeffs.squeeze()
print(coeffs.shape)

torch.Size([6])


In [38]:
mean_i = torch.mean(x_d,dim=0)
std_i = torch.std(x_d,dim=0)


In [39]:
x_d_norm = x_d - mean_i / std_i

In [40]:
x_d_norm = torch.nan_to_num(x_d_norm, nan=0.0)